In [1]:
from pyspark.sql import functions as F
# scan_df = df_scan
# rrm_radar_df = df_radar
from datetime import datetime

env = "staging"

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)
date_day = "2020-05-[1]*"
hr = "*" #now().hour-1

rrm_radar_path = s3_bucket + "rrm_radar_channel_stat/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)
print(rrm_radar_path)


date_day = datetime.today().strftime('%Y-%m-%d') #"2020-05-*"
curr_hr = '*'  # latest 
ap_neighbors_path = s3_bucket + "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

print(ap_neighbors_path)


scan_df = spark.read.csv(ap_neighbors_path, header=True)                   
rrm_radar_df = spark.read.csv(rrm_radar_path, header=True)    

scan_df.show(2)
rrm_radar_df.show(2)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1590100141701_0009,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
s3://mist-aggregated-stats-staging/aggregated-stats/rrm_radar_channel_stat/dt=2020-05-[1]*/hr=*/*.csv
s3://mist-aggregated-stats-staging/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-05-22/hr=*/*.csv
+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|12c07c11-51a3-495...|5c5b350e4cb8|5c5b350eb3f2|   5|1590108852|-48.0|
|276c2659-d2ae-43e...|5c5b358ee097|        null|   5|1590108488|-35.0|
+--------------------+------------+------------+----+----------+-----+
only showing top 2 rows

+--------------------+------------+-----------+-------------------+---------------+----------+
|             site_id|       ap_id|pre_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-----------+-------------------+---------------+----------+
|c943d077-1b6

In [15]:
rrm_radar_df = rrm_radar_df.withColumnRenamed("pre_channel", "radar_channel")
rrm_radar_df.show(3)

+--------------------+------------+-------------+-------------------+---------------+----------+
|             site_id|       ap_id|radar_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-------------+-------------------+---------------+----------+
|c943d077-1b67-491...|5c5b35500622|          104|                  1|     1589096727|1589094000|
|c943d077-1b67-491...|5c5b35500924|          116|                  1|     1589095299|1589094000|
|c943d077-1b67-491...|5c5b35500686|          104|                  1|     1589095247|1589094000|
+--------------------+------------+-------------+-------------------+---------------+----------+
only showing top 3 rows

# Radar_last_seen_in_site

In [16]:
# last seen in site

last_seen_in_site_df = rrm_radar_df.select('site_id', 'radar_channel', 'last_radar_time')\
                                    .groupBy('site_id', 'radar_channel')\
                                    .agg(F.max('last_radar_time').alias('last_seen_in_site'))
last_seen_in_site_df.show(10)

+--------------------+-------------+-----------------+
|             site_id|radar_channel|last_seen_in_site|
+--------------------+-------------+-----------------+
|d23f418c-a5ab-4e3...|          100|       1589343203|
|e80749be-963c-4d2...|          100|       1589277749|
|c943d077-1b67-491...|          120|       1589236479|
|c943d077-1b67-491...|           64|       1589208491|
|20325077-2d75-4fa...|            0|       1589468730|
|e80749be-963c-4d2...|          132|       1589534658|
|96ca2c79-b7db-402...|            0|       1589724186|
|f733a5d7-5179-4df...|           52|       1589214561|
|c943d077-1b67-491...|          132|       1589436093|
|67970e46-4e12-11e...|           52|       1589144494|
+--------------------+-------------+-----------------+
only showing top 10 rows

In [17]:
# AP's primary hit

In [18]:
rrm_radar_primary_df = rrm_radar_df\
                                .groupBy('site_id', 'ap_id', 'radar_channel')\
                                .agg(F.sum('radar_channel_count').alias('radar_channel_count'),
                                 F.max('last_radar_time').alias('last_seen'))
rrm_radar_primary_df.show(10)

+--------------------+------------+-------------+-------------------+----------+
|             site_id|       ap_id|radar_channel|radar_channel_count| last_seen|
+--------------------+------------+-------------+-------------------+----------+
|c943d077-1b67-491...|5c5b35500609|          108|                1.0|1589239583|
|c943d077-1b67-491...|5c5b3550082a|          100|                1.0|1589092454|
|f733a5d7-5179-4df...|5c5b353eec45|           52|                1.0|1589214561|
|35670395-458f-4ec...|5c5b350ec479|          132|                1.0|1589836761|
|c943d077-1b67-491...|5c5b35500587|          104|                1.0|1589175184|
|d7c802c4-1d3a-4aa...|5c5b350e0951|            0|                2.0|1589835231|
|c943d077-1b67-491...|5c5b35500587|           52|                1.0|1589210426|
|67970e46-4e12-11e...|5c5b353e4f10|          124|                2.0|1589918847|
|c943d077-1b67-491...|5c5b35500730|           56|                1.0|1589079621|
|c943d077-1b67-491...|5c5b35

# AP's neighbor impact

In [19]:
#  ap-ap2 neighbor relationship

scan_agg_df=  scan_df.groupBy('site', 'ap', 'ap2').agg(F.max('rssi').alias("max_rssi"),
                                                     F.count('ap2').alias('scan_counts')) 
scan_agg_df.show(2) 

+--------------------+------------+------------+--------+-----------+
|                site|          ap|         ap2|max_rssi|scan_counts|
+--------------------+------------+------------+--------+-----------+
|0b77b450-77ec-41f...|5c5b350e141e|5c5b350e0262|   -67.0|          1|
|3c9ac798-7ceb-487...|5c5b350e0569|5c5b353f574d|   -67.0|          4|
+--------------------+------------+------------+--------+-----------+
only showing top 2 rows

In [20]:
# inner-join with rrm-radar_df
ap_neighbor_radar_2 = scan_agg_df\
         .filter('ap2 is not Null and max_rssi >-75')\
         .join(rrm_radar_primary_df, [scan_agg_df.site == rrm_radar_primary_df.site_id,
                                      scan_agg_df.ap2 == rrm_radar_df.ap_id], 'inner')

ap_neighbor_radar_2.show(5)

+--------------------+------------+------------+--------+-----------+--------------------+------------+-------------+-------------------+----------+
|                site|          ap|         ap2|max_rssi|scan_counts|             site_id|       ap_id|radar_channel|radar_channel_count| last_seen|
+--------------------+------------+------------+--------+-----------+--------------------+------------+-------------+-------------------+----------+
|6db15e18-bc94-11e...|5c5b3550036b|5c5b350e070d|   -47.0|          8|6db15e18-bc94-11e...|5c5b350e070d|          100|                1.0|1589389208|
|d7c802c4-1d3a-4aa...|5c5b350e0951|5c5b350ec285|   -26.0|          4|d7c802c4-1d3a-4aa...|5c5b350ec285|            0|                1.0|1589816815|
|d7c802c4-1d3a-4aa...|5c5b352e20a8|5c5b350ec285|   -35.0|          4|d7c802c4-1d3a-4aa...|5c5b350ec285|            0|                1.0|1589816815|
|d7c802c4-1d3a-4aa...|5c5b350ec285|5c5b350ebfe2|   -34.0|          8|d7c802c4-1d3a-4aa...|5c5b350ebfe2|   

In [5]:
ap_neighbor_radar_2.filter('pre_channel>0').show(5)

+--------------------+------------+------------+--------+-----------+--------------------+------------+-----------+-------------------+---------------+----------+
|                site|          ap|         ap2|max_rssi|scan_counts|             site_id|       ap_id|pre_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+------------+--------+-----------+--------------------+------------+-----------+-------------------+---------------+----------+
|6db15e18-bc94-11e...|5c5b3550036b|5c5b350e070d|   -47.0|          8|6db15e18-bc94-11e...|5c5b350e070d|        100|                  1|     1589389208|1589389200|
|57b2f891-09c1-4dc...|5c5b350e02b7|5c5b353e4f47|   -32.0|          4|57b2f891-09c1-4dc...|5c5b353e4f47|         52|                  1|     1589641690|1589641200|
|57b2f891-09c1-4dc...|5c5b350e02b7|5c5b353e4f47|   -32.0|          4|57b2f891-09c1-4dc...|5c5b353e4f47|         52|                  1|     1589248902|1589248800|
|57b2f891-09c1-4dc...|

In [21]:
# ap impacted by ap2's radar_channel
ap_neighbor_radar_3 = ap_neighbor_radar_2\
                    .filter('radar_channel>0')\
                    .select('site', 'ap', 'ap2', 'radar_channel', 'radar_channel_count', 'max_rssi')\
                     .groupBy('site', 'ap', 'ap2', 'radar_channel')\
                     .agg(F.max('max_rssi').alias("max_rssi"))
ap_neighbor_radar_3.show()

+--------------------+------------+------------+-------------+--------+
|                site|          ap|         ap2|radar_channel|max_rssi|
+--------------------+------------+------------+-------------+--------+
|6db15e18-bc94-11e...|5c5b3550036b|5c5b350e070d|          100|   -47.0|
|57b2f891-09c1-4dc...|5c5b350e02b7|5c5b353e4f47|          100|   -32.0|
|57b2f891-09c1-4dc...|5c5b350e02b7|5c5b353e4f47|           52|   -32.0|
|57b2f891-09c1-4dc...|5c5b353e4f4c|5c5b353e4f47|          100|   -25.0|
|57b2f891-09c1-4dc...|5c5b353e4f4c|5c5b353e4f47|           52|   -25.0|
|20325077-2d75-4fa...|5c5b350ee3a5|5c5b355004ec|          116|   -40.0|
|57b2f891-09c1-4dc...|5c5b353e4f47|5c5b350e02b7|          124|   -42.0|
|12c07c11-51a3-495...|5c5b350e4cb8|5c5b354e318d|          124|   -35.0|
|67970e46-4e12-11e...|5c5b353f574d|5c5b350ecfeb|          140|   -64.0|
|57b2f891-09c1-4dc...|5c5b35500820|5c5b350e02b7|          124|   -65.0|
|67970e46-4e12-11e...|5c5b350ecfeb|5c5b353f574d|           52|  

In [22]:
# aggregated impacted by ap2's radar_channel
ap_neighbor_radar_4 = ap_neighbor_radar_3.groupBy('site', 'ap',  'radar_channel')\
         .agg(F.collect_list('ap2').alias('neighbor_aps'), F.collect_list('max_rssi').alias('neighbor_rssis'))\
        .withColumnRenamed('radar_channel', 'neighbor_radar_channel')
ap_neighbor_radar_4.show()

+--------------------+------------+----------------------+--------------+--------------+
|                site|          ap|neighbor_radar_channel|  neighbor_aps|neighbor_rssis|
+--------------------+------------+----------------------+--------------+--------------+
|67970e46-4e12-11e...|5c5b350ecfeb|                    52|[5c5b353f574d]|       [-65.0]|
|20325077-2d75-4fa...|5c5b350ee3a5|                   116|[5c5b355004ec]|       [-40.0]|
|57b2f891-09c1-4dc...|5c5b353e4f47|                   124|[5c5b350e02b7]|       [-42.0]|
|57b2f891-09c1-4dc...|5c5b35500820|                   124|[5c5b350e02b7]|       [-65.0]|
|12c07c11-51a3-495...|5c5b350e4cb8|                   124|[5c5b354e318d]|       [-35.0]|
|57b2f891-09c1-4dc...|5c5b353e4f4c|                   100|[5c5b353e4f47]|       [-25.0]|
|6db15e18-bc94-11e...|5c5b3550036b|                   100|[5c5b350e070d]|       [-47.0]|
|57b2f891-09c1-4dc...|5c5b350e02b7|                   100|[5c5b353e4f47]|       [-32.0]|
|57b2f891-09c1-4dc...

In [28]:
# join for radar_last_seen_in_site
ap_neighbor_radar_5 = ap_neighbor_radar_4\
                .join(last_seen_in_site_df, [ap_neighbor_radar_4.site == last_seen_in_site_df.site_id, 
                 ap_neighbor_radar_4.neighbor_radar_channel==last_seen_in_site_df.radar_channel], 'inner')\
                .select('site', 'ap', 'radar_channel', 'neighbor_rssis', 'neighbor_aps', 'last_seen_in_site')

ap_neighbor_radar_5.show()

+--------------------+------------+-------------+--------------+--------------+-----------------+
|                site|          ap|radar_channel|neighbor_rssis|  neighbor_aps|last_seen_in_site|
+--------------------+------------+-------------+--------------+--------------+-----------------+
|67970e46-4e12-11e...|5c5b350ecfeb|           52|       [-65.0]|[5c5b353f574d]|       1589144494|
|20325077-2d75-4fa...|5c5b350ee3a5|          116|       [-40.0]|[5c5b355004ec]|       1589209277|
|57b2f891-09c1-4dc...|5c5b353e4f47|          124|       [-42.0]|[5c5b350e02b7]|       1589924617|
|57b2f891-09c1-4dc...|5c5b35500820|          124|       [-65.0]|[5c5b350e02b7]|       1589924617|
|12c07c11-51a3-495...|5c5b350e4cb8|          124|       [-35.0]|[5c5b354e318d]|       1589675016|
|57b2f891-09c1-4dc...|5c5b353e4f4c|          100|       [-25.0]|[5c5b353e4f47]|       1589459444|
|6db15e18-bc94-11e...|5c5b3550036b|          100|       [-47.0]|[5c5b350e070d]|       1589389208|
|57b2f891-09c1-4dc..

In [29]:

scan_final_df = ap_neighbor_radar_5
scan_final_df.show()

+--------------------+------------+-------------+--------------+--------------+-----------------+
|                site|          ap|radar_channel|neighbor_rssis|  neighbor_aps|last_seen_in_site|
+--------------------+------------+-------------+--------------+--------------+-----------------+
|67970e46-4e12-11e...|5c5b350ecfeb|           52|       [-65.0]|[5c5b353f574d]|       1589144494|
|20325077-2d75-4fa...|5c5b350ee3a5|          116|       [-40.0]|[5c5b355004ec]|       1589209277|
|57b2f891-09c1-4dc...|5c5b353e4f47|          124|       [-42.0]|[5c5b350e02b7]|       1589924617|
|57b2f891-09c1-4dc...|5c5b35500820|          124|       [-65.0]|[5c5b350e02b7]|       1589924617|
|12c07c11-51a3-495...|5c5b350e4cb8|          124|       [-35.0]|[5c5b354e318d]|       1589675016|
|57b2f891-09c1-4dc...|5c5b353e4f4c|          100|       [-25.0]|[5c5b353e4f47]|       1589459444|
|6db15e18-bc94-11e...|5c5b3550036b|          100|       [-47.0]|[5c5b350e070d]|       1589389208|
|57b2f891-09c1-4dc..

In [27]:
# ap_neighbor_radar_22 = scan_df_2\
#          .filter('ap2 is not Null and max_rssi >-75')\
#          .join(rrm_radar_df, [scan_df_2.site == rrm_radar_df.site_id, scan_df_2.ap2 == rrm_radar_df.ap_id], 'inner')\
#          .select('site', 'ap', 'ap2', 'pre_channel', 'radar_channel_count', 'max_rssi')\
#          .groupBy('site', 'ap', 'ap2', 'pre_channel')\
#          .agg(F.max('max_rssi').alias("max_rssi"))\
#         .groupBy('site', 'ap', 'pre_channel')\
#          .agg(F.collect_list('ap2').alias('neighbor_aps'), F.collect_list('max_rssi').alias('neighbor_rssis'))\
#         .withColumnRenamed('pre_channel', 'neighbor_radar_channel')                                                                                          

# ap_neighbor_radar_22.show()  

In [26]:
# ap_neighbor_radar_2.select('ap', 'neighbor_radar_channel', 'neighbor_aps', 'neighbor_rssis').show(2, False)  

In [11]:

# Goal: 

#rrm/policy/org/{org}/site/{site}/ap/ { "116" : {"weight_tot": 0.8, "radar_detected":0.5, "neighbor_radar": 0.3, "geolocation": 0.0, "last_seen": 11111}}